Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/explain-model/azure-integration/remote-explanation/explain-model-on-amlcompute.png)

# Train and explain models remotely via Azure Machine Learning Compute


_**This notebook showcases how to use the Azure Machine Learning Interpretability SDK to train and explain a regression model remotely on an Azure Machine Leanrning Compute Target (AMLCompute).**_




## Table of Contents

1. [Introduction](#Introduction)
1. [Setup](#Setup)
    1. Initialize a Workspace
    1. Create an Experiment
    1. Introduction to AmlCompute
    1. Submit an AmlCompute run in a few different ways
        1. Option 1: Provision as a run based compute target 
        1. Option 2: Provision as a persistent compute target (Basic)
        1. Option 3: Provision as a persistent compute target (Advanced)
1. Additional operations to perform on AmlCompute
1. [Download model explanations from Azure Machine Learning Run History](#Download)
1. [Visualize explanations](#Visualize)
1. [Next steps](#Next)

## Introduction

This notebook showcases how to train and explain a regression model remotely via Azure Machine Learning Compute (AMLCompute), and download the calculated explanations locally for visualization.
It demonstrates the API calls that you need to make to submit a run for training and explaining a model to AMLCompute, download the compute explanations remotely, and visualizing the global and local explanations via a visualization dashboard that provides an interactive way of discovering patterns in model predictions and downloaded explanations.

We will showcase one of the tabular data explainers: TabularExplainer (SHAP).

Problem: Boston Housing Price Prediction with scikit-learn (train a model and run an explainer remotely via AMLCompute, and download and visualize the remotely-calculated explanations.)

![](./images/explanations-run-history.png)

## Setup

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize a Workspace

Connect to the workspace

In [ ]:
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## Create An Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
from azureml.core import Experiment
experiment_name = 'explainer-remote-run-on-amlcompute'
experiment = Experiment(workspace=ws, name=experiment_name)

## Introduction to AmlCompute

Azure Machine Learning Compute is managed compute infrastructure that allows the user to easily create single to multi-node compute of the appropriate VM Family. It is created **within your workspace region** and is a resource that can be used by other users in your workspace. It autoscales by default to the max_nodes, when a job is submitted, and executes in a containerized environment packaging the dependencies as specified by the user. 

Since it is managed compute, job scheduling and cluster management are handled internally by Azure Machine Learning service. 

For more information on Azure Machine Learning Compute, please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)

**Note**: As with other Azure services, there are limits on certain resources (for eg. AmlCompute quota) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.



## Submit an AmlCompute run in a few different ways

First lets check which VM families are available in your region. Azure is a regional service and some specialized SKUs (especially GPUs) are only available in certain regions. Since AmlCompute is created in the region of your workspace, we will use the supported_vms () function to see if the VM family we want to use ('STANDARD_D2_V2') is supported.

You can also pass a different region to check availability and then re-create your workspace in that region through the [configuration notebook](../../../configuration.ipynb)

### Create project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on

In [ ]:
import os
import shutil

project_folder = './explainer-remote-run-on-amlcompute'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train_explain.py', project_folder)

Note: if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).

### Fetch or create the compute target 

We are going to use the compute target you had created before (make sure you provide the same name here in the variable `cpu_cluster_name`. 

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Reference your existing cluster
cpu_cluster_name = "cpu-cluster"

cpu_cluster = ws.compute_targets[cpu_cluster_name]

The above code will throw an error if you didn't set the cluster up in the first part of the workshop. In that case, you can provision a cluster here by executing the next cell.

**Note:** By default a cluster autoscales from 0 nodes and provisions dedicated VMs to run your job in a container. This is useful when you want to continously re-use the same target, debug it between jobs or simply share the resource with other users of your workspace.

* `vm_size`: VM family of the nodes provisioned by AmlCompute. Simply choose from the supported_vmsizes() above
* `max_nodes`: Maximum nodes to autoscale to while running a job on AmlCompute

In [ ]:
# If you didn't create the cluster in the first part of the workshop, this code will create it
#compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
#                                                       max_nodes=8, 
#                                                       idle_seconds_before_scaledown=7200)
#cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
#cpu_cluster.wait_for_completion(show_output=True)

### Configure & Run

In [ ]:
from azureml.train.estimator import Estimator

pip_packages = [
                'azureml-defaults==1.0.76', 'azureml-core==1.0.76', 'azureml-telemetry==1.0.76',
                'azureml-dataprep==1.1.31', 'joblib==0.14.0', 'sklearn-pandas==1.7.0', 'pandas==0.23.4',
                'azureml-contrib-interpret'
               ]

estimator = Estimator(source_directory=project_folder, 
                      compute_target=cpu_cluster,
                      entry_script='train_explain.py',
                      pip_packages=pip_packages,
                      conda_packages=['scikit-learn==0.20.3'],
                      inputs=[ws.datasets['IBM-Employee-Attrition'].as_named_input('attrition')])

run = experiment.submit(estimator)
run

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

![](images/aml-run.png)

## (OPTIONAL) Additional operations to perform on AmlCompute Cluster

You can perform more operations on AmlCompute such as updating the node counts or deleting the compute. 

In [ ]:
# Get_status () gets the latest status of the AmlCompute target
cpu_cluster.get_status().serialize()

In [ ]:
# The update() call takes in the min_nodes, max_nodes and idle_seconds_before_scaledown and updates the AmlCompute target
# cpu_cluster.update(min_nodes=1)
# cpu_cluster.update(max_nodes=10)
#cpu_cluster.update(idle_seconds_before_scaledown=7200)
# cpu_cluster.update(min_nodes=2, max_nodes=4, idle_seconds_before_scaledown=600)

In [ ]:
# Delete() is used to deprovision and delete the AmlCompute target. Useful if you want to re-use the compute name 
# 'cpu-cluster' in this case but use a different VM family for instance.

# cpu_cluster.delete()

In [ ]:
run.wait_for_completion(show_output=True)

## Download 
### 1. Download model explanation data.

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

# Get model explanation data
client = ExplanationClient.from_run(run)
global_explanation = client.download_model_explanation()

In [ ]:
# Get the top k (e.g., 4) most important features with their importance values
global_explanation_topk = client.download_model_explanation(top_k=4)
global_importance_values = global_explanation_topk.get_ranked_global_values()
global_importance_names = global_explanation_topk.get_ranked_global_names()

In [ ]:
print('global importance values: {}'.format(global_importance_values))
print('global importance names: {}'.format(global_importance_names))

## See model's files

In [ ]:
print(run.get_file_names())

### 2. Download model and test set files

In [ ]:
# retrieve model for visualization and deployment
from azureml.core.model import Model
import joblib

# Download test dataset file
run.download_file('x_test_ibm.pkl')
x_test = joblib.load('x_test_ibm.pkl')

# Download trained model
run.download_file('original_model.pkl')
original_model = joblib.load('original_model.pkl')

original_model

## Visualize
Load the visualization dashboard (currently broken)

In [ ]:
from interpret_community.widget import ExplanationDashboard

In [ ]:
ExplanationDashboard(global_explanation, original_model, datasetX=x_test)